# Advanced OOP
Here, I follow the LinkedIn Learning course [Advanced Python: Object-Oriented
Programming](https://www.linkedin.com/learning/advanced-python-object-oriented-programming/advanced-object-oriented-programming-oop?resume=false&u=72605090)
by Miki Tebeka and try the code. Here is the [course
repo](https://github.com/LinkedInLearning/advanced-python-object-oriented-programming-4510177)
(it is very good). I have added quite some additional explanations from ChatGPT, that I
needed to understand, and also some additional exercises.

## Part 2: Methods

### Using class methods

- mostly used to as a different way to create an instance of a class
    - for instance see the many ways to instantiate pythons [datetime objects](https://docs.python.org/3/library/datetime.html#datetime-objects)

In [ ]:
class Auth:
    def __init__(self, db):
        self.db = db

    def from_token(self, token):
        return self.db.get(token)

# that's our database
auth = Auth({
    'b92d877': 'carly',
    '18317ac': 'elliot',
})


class User:
    def __init__(self, login):
        self.login = login
        # TODO: More fields

    @classmethod # as alternate constructor
    def from_token(cls, token):
        login = auth.from_token(token)
        return cls(login)

class Admin(User):
    ...  # TODO
    

u = User('carly')
print(u.login, type(u))

u = User.from_token('b92d877')
print(u.login, type(u))

a = Admin.from_token('18317ac')
print(a.login, type(a))

carly <class '__main__.User'>
carly <class '__main__.User'>
elliot <class '__main__.Admin'>


### Using static methods

- functions that are assigned to a class
- only reason: discoverability

In [8]:
from random import choice

adjectives = ['cool', 'funny', 'strong']
names = ['bruce', 'carol', 'natasha']


class VM:
    def __init__(self):
        self.name = VM.random_name()


    @staticmethod
    def random_name():
        adjective, name = choice(adjectives), choice(names)
        return f'{adjective}_{name}'


# %% Test
vm = VM()
print(vm.name)

strong_bruce


### Mixin classes

- add functionality to other classes

In [9]:
import logging


logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s: %(message)s',
)

class LoggerMixin:
    def log_id(self):
        logging.info('%s with id %r', self.name, self.id)


class User:
    def __init__(self, name, id):
        self.name = name
        self.id = id


class VM:
    def __init__(self, name, id):
        self.name = name
        self.id = id

class LoggedUser(LoggerMixin, User):
    pass

class LoggedVM(LoggerMixin, VM):
    pass


user = LoggedUser('root', 1)
user.log_id()

vm = LoggedVM('m1', '4922a77')
vm.log_id()

INFO: root with id 1
INFO: m1 with id '4922a77'


### Abstract base classes

- this is to discover errors (here: typo in method name) as early as possible

- using an abstract base class makes the error discoverable in build time rather than in
  test time
    - (I still think this improvement in error discovery is minimal though)

- very practical with mixin classes, because it can be specified what the mixin expects
  from the lass it enhances

In [10]:
from abc import ABC, abstractmethod


class Plugin(ABC):
    @abstractmethod
    def notify(self, event):
        pass

    @abstractmethod
    def shutdown(self):
        pass


class LoggingPlugin(Plugin):
    def notify(self, event):
        print(f'got {event}')

    def shutdown(self):
        print('logger shutting down')


class SecurityPlugin(Plugin):
    def notify(self, event):
        if event.action == 'login' and event.user == 'elliot':
            print(f'WARNING: {event.user} has logged in')

    def shutdwon(self): ######## watch the typo
        print('security shutting down')


def notify(plugins, event):
    for plugin in plugins:
        plugin.notify(event)


def shutdown(plugins):
    for plugin in plugins:
        plugin.shutdown()



class Event:
    def __init__(self, user, action):
        self.user = user
        self.action = action

plugins = [LoggingPlugin(), SecurityPlugin()]
event = Event('elliot', 'login')
notify(plugins, event)
shutdown(plugins)

TypeError: Can't instantiate abstract class SecurityPlugin without an implementation for abstract method 'shutdown'

In [11]:
# the mixin class specifies what methods are expected in the class it enhances

from abc import ABC, abstractmethod

class SaveMixin(ABC):
    @abstractmethod
    def serialize(self):
        pass

    def save(self, filepath):
        data = self.serialize()
        with open(filepath, 'w') as f:
            f.write(data)


### Defining interfaces with `typing.Protocol`

In [ ]:
from typing import Protocol


class Writer(Protocol):
    # that's only for defining the signatures of the functions and the expected types
    def write(self, data: bytes) -> None:
        ... # code doesn't matter


import json

def store_json(w: Writer, obj: dict) -> None:
    # expects an object adhering to the Writer protocol passed as `w`
    data = json.dumps(obj).encode('utf-8') # sends bytes to the object passed as `w`
    w.write(data)


class S3File:
    # does not inherit from Writer, but would be expected to adher to it
    def write(self, data: str) -> None: # type mismatch! expects str, but gets bytes; would raise when running mypy
        print(f's3: write: {data!r}')


out = S3File()
obj = {
    'id': '007',
    'lat': 51.4871871,
    'lng': -0.1270605,
}
store_json(out, obj) 
# `store_json` gets passed an object not adhering to the Writer protocol, but still
# treats it as if it was, printing the data in the write method as bytes (which shows by
# the `b'` in front of the dict: it’s treating the bytes as a string).

s3: write: b'{"id": "007", "lat": 51.4871871, "lng": -0.1270605}'


## Part 3: Special Methods

### String representations
- `__repr__`: representation for developers
- `__str__`: for print()
- `__format__`: formattable version of `__str__`

In [ ]:
import re


class Payment:
    def __init__(self, amount, currency):
        self.amount = amount
        self.currency = currency

    def __str__(self):
        return f'{self.currency}{self.amount:.2f}'

    def __repr__(self):
        name = self.__class__.__name__
        return f'{name}({self.amount!r}, {self.currency!r})' # `!r` stands for repr

    def _replace(self, match):
        ### customises the formatting codes, that can be used in f-strings or format() function
        if match[1] == 'a':
            return f'{self.amount:.2f}'
        if match[1] == 'c':
            return self.currency
        raise ValueError(f'unknown format: {match.group()}')

    def __format__(self, spec):
        if not spec:
            return str(self)
        return re.sub(r'(?<!%)%([ac])', self._replace, spec)



p = Payment(123.45, '£')

# %% str
print(p)

# %% repr
print(f'p={p!r}')
print(f'{p=}')
p # Payment(123.45, '£') --> also returns the repr

# %% format
print(f'A payment of {p:%a} in {p:%c}')
# `%a` is a custom placeholder for amount and `%c` is a custom placeholder for currency

£123.45
p=Payment(123.45, '£')
p=Payment(123.45, '£')
A payment of 123.45 in £


Payment(123.45, '£')

### Sequences
- [Collections.abc doc page](https://docs.python.org/3/library/collections.abc.html) shows that every type that wants to emulate a python Sequence class, needs to implement `__getitem__` and `__len__` as abstract methods, but also `__contains__`, `__iter__`, `__reversed__`, `index`, and `count` as mixin methods
    - where the abstract methods are the minimum methods we must implement in our class for it to be considered a valid subclass of `Sequence`
    - mixin methods are provided automatically by the ABC if the required abstract methods are implemented
- using `collections.abc.Sequence` helps us to emulate a sequence-type class; without it we would have to code the mixin methods and also register our class as a virtual subclass of the python Sequence class

In [ ]:
from collections.abc import Sequence

class Node:
    def __init__(self, value, next):
        self.value = value
        self.next = next


class Stack(Sequence):
    def __init__(self):
        self._head = None

    def push(self, value):
        self._head = Node(value, self._head)

    def pop(self):
        if self._head is None:
            raise ValueError('pop from empty stack')

        value = self._head.value
        self._head = self._head.next
        return value

    # until here, it was a regular Stack
    # now we make it a Sequence:
    def __len__(self):
        # counting number of items
        count = 0
        node = self._head
        while node:
            count += 1
            node = node.next
        return count

    def __getitem__(self, index):
        # called when we trigger `obj[num]`
        node = self._head
        while index > 0 and node:
            # iterating backwards, because this is how a stack is made
            # it's also accessing the items backwards looking from the outside perspective
            index -= 1
            node = node.next
        if not node:
            raise IndexError(index)
        return node.value


s = Stack()
for c in 'Python':
    s.push(c)
print('len:', len(s))
print('s[2]:', s[2]) # like accessing list_type[-2]
print('t' in s) # __contains__ is provided as a mixin method automatically

len: 6
s[2]: h
True


In [ ]:
# experimenting with regular Stack

class Node:
    def __init__(self, value, next):
        self.value = value
        self.next = next


class Stack():
    def __init__(self):
        self._head = None

    def push(self, value):
        # set one node value
        self._head = Node(value, self._head)

    def pop(self):
        # re-winds the latest call to `push()`
        if self._head is None:
            raise ValueError('pop from empty stack')

        value = self._head.value
        self._head = self._head.next
        return value
    
stack = Stack()
print(stack._head)
# None

stack.push(1)
print(f"{stack._head} with {stack._head.__dict__}")
# <__main__.Node object at 0x721dd01bbd10> with {'value': 1, 'next': None}

stack.push(2) # the former node becomes the `next` value
print(f"{stack._head} with {stack._head.__dict__}")
# <__main__.Node object at 0x721dd01b9ac0> with {'value': 2, 'next': <__main__.Node object at 0x721dd01bbd10>}

popped_value = stack.pop() # pops most recently added value
print(popped_value) # to print returned value
# 2

print(f"{stack._head} with {stack._head.__dict__}")
# <__main__.Node object at 0x721db2f769c0> with {'value': 1, 'next': None} 

popped_value = stack.pop() # pops most recently added value
print(popped_value) # to print returned value
# 1

None
<__main__.Node object at 0x721db2f769c0> with {'value': 1, 'next': None}
<__main__.Node object at 0x721db2f770e0> with {'value': 2, 'next': <__main__.Node object at 0x721db2f769c0>}
2
<__main__.Node object at 0x721db2f769c0> with {'value': 1, 'next': None}
1


### Mappings
- [Collections.abc doc page](https://docs.python.org/3/library/collections.abc.html)
  shows that every type that wants to emulate a python Mapping class (similar to a
  `dict`, but MutableMapping is even more like a `dict`), needs to implement
  `__getitem__` `__iter__` and `__len__` as abstract methods, and other methods are
  provided as mixin methods

In [ ]:
from collections.abc import Mapping

class Headers(Mapping):
    def __init__(self, headers: dict):
        self._headers = {
            key.lower(): value
            for key, value in headers.items()
        }

    def __len__(self):
        return len(self._headers)

    def __getitem__(self, key):
        key = key.lower()
        return self._headers[key]

    def __iter__(self):
        return iter(self._headers)


headers = Headers({
    'Content-Type': 'application/json; charset=utf-8',
    'Content-Length': '1366',
    'Accept-Ranges': 'bytes',
})
print(len(headers), 'headers') # calls out `__len__`
print('Content Type:', headers['content-type']) # calls our `__getitem__`

# both call our `__iter__`
for key in headers:
    print('key:', key)
for key, value in headers.items():
    print(key, '->', value)

3 headers
Content Type: application/json; charset=utf-8
key: content-type
key: content-length
key: accept-ranges
content-type -> application/json; charset=utf-8
content-length -> 1366
accept-ranges -> bytes


- for Mapping types, there is also `__missing__` that get's called when a key cannot be
  found:

In [20]:
class TraceIDs(dict):
    # inheriting from dict, because it's more straightforward for this example than to
    # use `Collections.abc.MutableMapping`
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        self._counter = 1

    def __missing__(self, key):
        val = self._counter
        self._counter += 1
        self[key] = val # set the counter value to that missing key
        return val


trace_ids = TraceIDs()
print('calls ID:', trace_ids['http.calls'])
print('calls ID:', trace_ids['http.calls'])
print('errors ID:', trace_ids['http.errors'])


# same:
from collections import defaultdict
from itertools import count

trace_ids = defaultdict(count(1).__next__)
print('calls ID:', trace_ids['http.calls'])
print('calls ID:', trace_ids['http.calls'])
print('errors ID:', trace_ids['http.errors'])

calls ID: 1
calls ID: 1
errors ID: 2
calls ID: 1
calls ID: 1
errors ID: 2


I understand the usecase of this is to store miss-aligned http requests with an
corresponding ID that increments with every not found key. Maybe for better analysing
error afterwards, I imagine.

### Numbers
- see [Emulating numeric
  types](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types) for
  all the methods that need to be implemented to emulate a number, for instance `__add__`, `__truediv__`, or `__pow__`, but also in revised order such as `__radd__`, and so on

In [21]:
class Duration:
    unit_values = {
        'ns': 1,
        'us': 1000,
        'ms': 1_000_000,
    }

    def __init__(self, value: float, unit: str):
        if value < 0 or unit not in Duration.unit_values:
            raise ValueError(f'invalid duration: {value=}, {unit=}')

        self.value = value
        self.unit = unit

    def __repr__(self):
        return f'{self.value}{self.unit}'

    def __add__(self, other):
        v1 = self.value * Duration.unit_values[self.unit]
        v2 = other.value * Duration.unit_values[other.unit]
        value = (v1 + v2) / Duration.unit_values[self.unit]
        return Duration(value, self.unit)


u1 = Duration(317, 'us')
u2 = Duration(2.7, 'ms')
print(u1 + u2)

3017.0us


### Callable
